In [4]:
import sys
import os
import shutil
import warnings
import torch
import numpy as np
import random
from matplotlib import pyplot as plt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')
else:
  print("cuda is available")

/tmp/ipykernel_26749/286268813.py:13: UserWarning: CUDA is not available.
  warnings.warn('CUDA is not available.')


In [1]:
#@title Mount your Google Drive
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
folder = "/content/gdrive/MyDrive/ELE6310_project/" #@param {type:"string"}

!ln -Ts $folder /content/A1 2> /dev/null

# Add the assignment folder to Python path
if '/content/gdrive/MyDrive/ELE6310_project/' not in sys.path:
    sys.path.insert(0, '/content/gdrive/MyDrive/ELE6310_project/')

In [2]:
# Install requirements
!pip install -qr requirements.txt

In [3]:
from common.utils import load_CIFAR10_dataset, evaluate, fit, model_size
#from common.resnet import resnet20
from common.resnet_brevitas import resnet20

In [5]:
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

In [6]:
train_loader, test_loader, calibration_loader = load_CIFAR10_dataset(batch_size=256, calibration_batch_size=1024)
model = resnet20(pretrained=True, save_path='./save/')
device = torch.device('cuda:0')
model.to(device)

accuracy = evaluate(model, test_loader, device)
print("test accuracy of fp model:", accuracy, "model size:", model_size(model))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet20-4118986f.pt" to ./save/cifar10_resnet20-4118986f.pt


  0%|          | 0.00/1.09M [00:00<?, ?B/s]

test accuracy of fp model: 92.5999984741211 model size: model size: 1.046MB


In [7]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
scheduler = None
train_accuracy, test_accuracy = fit(model, 5, train_loader, test_loader, criterion, optimizer, scheduler, device)

fit:  20%|██        | 1/5 [00:47<03:08, 47.05s/it]

epoch 1: train_accuracy=99.90797424316406, test_accuracy=92.42999267578125


fit:  40%|████      | 2/5 [01:34<02:21, 47.27s/it]

epoch 2: train_accuracy=99.9181900024414, test_accuracy=92.4699935913086


fit:  60%|██████    | 3/5 [02:21<01:34, 47.06s/it]

epoch 3: train_accuracy=99.91205596923828, test_accuracy=92.44999694824219


fit:  80%|████████  | 4/5 [03:11<00:48, 48.34s/it]

epoch 4: train_accuracy=99.91614532470703, test_accuracy=92.53999328613281


epoch 5: train_accuracy=99.92228698730469, test_accuracy=92.4699935913086
